In [1]:
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
output_dir = '/mnt/share/costeffectiveness/results/vivarium_ciff_sam/v7.0.0_full_model_no_x_factor/ciff_sam/2022_03_04_17_34_48/count_data/'

In [3]:
pd.read_csv(output_dir + 'wasting_transition_count.csv').age.unique()

array(['early_neonatal', 'late_neonatal', '1-5_months', '6-11_months',
       '12_to_23_months', '2_to_4'], dtype=object)

In [4]:
scenarios = ['baseline',
             'sam_treatment',
             'mam_treatment',
             'sqlns',
             'lbwsg_interventions',
             'zinc_supplementation']
scenario_names = ['Baseline',
                  'SAM treatment',
                 'MAM treatment',
                 'SQ-LNS',
                 'LBWSG interventions',
                 'Zinc supplementation']
scenario_colors = ['tab:blue',
                  'tab:orange',
                  'tab:green',
                  'tab:red',
                  'tab:purple',
                  'tab:brown']
scenario_order = pd.DataFrame()
scenario_order['scenario'] = scenarios
scenario_order['order'] = list(range(0,len(scenarios)))
scenario_order

,scenario,order
0,baseline,0
1,sam_treatment,1
2,mam_treatment,2
3,sqlns,3
4,lbwsg_interventions,4
5,zinc_supplementation,5


In [5]:
def calculate_wasting_values(output_dir, ages, strata_cols, measure, metric, 
                             relative_scenario):
    if measure=='prevalence':
        data = pd.read_csv(output_dir + 'wasting_state_person_time.csv').drop(columns=['Unnamed: 0'])
        wasted = data.loc[data.cause.str.contains('acute_malnutrition')]
        wasted['cause'] = 'wasted'
        wasted = (wasted.groupby([c for c in wasted.columns if c != 'value']).sum())
        data = pd.concat([data,wasted.reset_index()],ignore_index=True)
    elif measure=='incidence':
        data = (pd.read_csv(output_dir + 'wasting_transition_count.csv')
                .drop(columns=['Unnamed: 0']).rename(columns={'measure':'cause'}))
        wasted = data.loc[data.cause.isin(['mild_child_wasting_to_moderate_acute_malnutrition_event_count',
                                          'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count'])]
        wasted['cause'] = 'wasted'
        total_mam = data.loc[data.cause.isin(['mild_child_wasting_to_moderate_acute_malnutrition_event_count',
                                             'severe_acute_malnutrition_to_moderate_acute_malnutrition_event_count'])]
        total_mam['cause'] = 'total_mam'
        wasted = pd.concat([wasted,total_mam])
        wasted = (wasted.groupby([c for c in wasted.columns if c != 'value']).sum())
        data = pd.concat([data,wasted.reset_index()],ignore_index=True)
        pt = pd.read_csv(output_dir + 'wasting_state_person_time.csv').drop(columns=['Unnamed: 0'])
        groupby_cols = ['input_draw','scenario','cause']
    data = data.loc[data.age.isin(ages)]
    if 'year' (strata_cols)
    data = data.groupby(['input_draw','scenario'] + strata_cols + ['cause']).sum()
    if measure=='prevalence':
        data = data / data.groupby(['input_draw','scenario'] + strata_cols).sum() * 100
    else:
        pt = pt.loc[pt.age.isin(ages)].groupby(['input_draw','scenario'] + strata_cols).sum()
        data = data / pt * 100_000     
    #if metric=='value':
    #    data = (data.groupby(['scenario','cause'] + strata_cols)
    #                .describe(percentiles=[0.025,0.975]))
    if metric != 'value':
        data_baseline = (data.reset_index().loc[data.reset_index().scenario==relative_scenario]
                   .set_index(['input_draw','cause'] + strata_cols)
                   .drop(columns='scenario'))
        if metric=='percent_reduction':
            data = ((data_baseline - data.reset_index().set_index(['input_draw','cause'] + strata_cols + ['scenario']))
                    / data_baseline) * 100
        elif metric=='absolute_reduction':
            data = data_baseline - data.reset_index().set_index(['input_draw','cause'] + strata_cols + ['scenario'])
            
        #data = (data.reset_index().groupby(['scenario','cause'] + strata_cols)
        #        .describe(percentiles=[0.025,0.975])).drop(columns='input_draw')
    data = data.reset_index().merge(scenario_order, on='scenario').sort_values(by='order')
    data = (data.groupby(['scenario','cause'] + strata_cols)
                    .describe(percentiles=[0.025,0.975]))[['value']]
    return data.reset_index()

SyntaxError: invalid syntax (<ipython-input-5-eb0166124ce8>, line 24)

In [ ]:
strata_cols = ['year']

In [ ]:
'year' in (strata_cols)

In [ ]:
def format_2026_table(measure, metric, relative_scenario):
    t = calculate_wasting_values(output_dir, ages, ['year'], measure, metric, relative_scenario)
    t = t.loc[(t.year==2026)]
    if measure=='incidence':
        t = t.loc[(t.cause.isin(['mild_child_wasting_to_moderate_acute_malnutrition_event_count',
                            'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count']))]
    t['result'] = (t['value','mean'].round(1).astype(str)
                  + ' ('
                  + t['value','2.5%'].round(1).astype(str)
                  + ', '
                  + t['value','97.5%'].round(1).astype(str)
                  + ')')
    t = t.set_index(['scenario','cause']).unstack()[['result']].reset_index()
    t['scenario_order'] = np.where(t.scenario=='baseline',0,
                                  np.where(t.scenario=='sam_treatment',1,
                                          np.where(t.scenario=='mam_treatment',2,
                                                  np.where(t.scenario=='sqlns',3,
                                                          np.where(t.scenario=='lbwsg_interventions',4,5)))))
    t = t.sort_values(by='scenario_order').drop(columns='scenario_order')
    return t

In [ ]:
six_to_59_ages = ['6-11_months',
       '12_to_23_months', '2_to_4']
ages = ['6-11_months',
       '12_to_23_months', '2_to_4']

## Finding 1: cross sectional prevalence

In [ ]:
format_2026_table('prevalence', 'value', 'baseline')

In [ ]:
format_2026_table('prevalence', 'absolute_reduction', 'baseline')

In [ ]:
causes = ['wasted','moderate_acute_malnutrition','severe_acute_malnutrition']
cause_titles = ['Acute malnutrition\n(MAM and SAM)\n',
               'Moderate acute\nmalnutrition\n',
               'Severe acute\nmalnutrition\n']
metrics = ['value','percent_reduction']

fig, axs = plt.subplots(len(metrics), len(causes),dpi=250)
for metric in metrics:
    data = calculate_wasting_values(output_dir, six_to_59_ages, ['year'], 
                                        'prevalence', 
                                        metric, 
                                        'baseline')
    for cause in causes:
        p = data.loc[data.cause==cause]
        pos = axs[metrics.index(metric),causes.index(cause)]   
        for i in list(range(0,len(scenarios))):
            p_sub = p.loc[p.scenario==scenarios[i]]
            pos.plot(p_sub['year'], p_sub['value','mean'], marker='o', label=scenario_names[i])
            pos.fill_between(p_sub['year'], p_sub['value','2.5%'], p_sub['value','97.5%'], alpha=0.2)
        pos.grid()
        pos.set_xticks(p_sub.year.unique())
        pos.set_xticklabels(p_sub.year.unique(),fontsize=15)
        pos.tick_params(axis='y', labelsize=15)
        if metrics.index(metric)==0:
            pos.set_ylim(0,p.loc[p.scenario=='baseline']['value','97.5%'].max()*1.1)
            pos.set_title(cause_titles[causes.index(cause)],size=20)
            if causes.index(cause)==0:
                pos.set_ylabel('Percent exposed\n',size=20)
                pos.legend(#['Baseline','+Wasting Treatment','+SQ-LNS','+Zinc supplementation'], 
                       fontsize=15)
        if metric=='percent_reduction':
            pos.set_ylim(-2,50)
            if causes.index(cause)==0:
                pos.set_ylabel('Percent reduction\nin exposure\nrelative to baseline\n',size=20)

plt.subplots_adjust(right=len(causes), top=len(metrics))
#plt.legend(['Baseline','Alternative scenario 1','Alternative scenario 2'], bbox_to_anchor=[0.25,4.15], fontsize=20)
plt.suptitle('Simulated intervention impacts on acute malnutrition\ncross-sectional exposure among children six to 59 months\n(with 95% uncertainty intervals)',
             y=2.7,x=1.5,size=25)

In [ ]:
causes = ['wasted','moderate_acute_malnutrition','severe_acute_malnutrition']
cause_titles = ['Acute malnutrition\n(MAM and SAM)\n',
               'Moderate acute\nmalnutrition\n',
               'Severe acute\nmalnutrition\n']
metrics = ['value','percent_reduction']

fig, axs = plt.subplots(len(metrics), len(causes),dpi=250)
for metric in metrics:
    data = calculate_wasting_values(output_dir, ['6-11_months'], ['year'], 
                                        'prevalence', 
                                        metric, 
                                        'baseline')
    for cause in causes:
        p = data.loc[data.cause==cause]
        pos = axs[metrics.index(metric),causes.index(cause)]   
        for i in list(range(0,len(scenarios))):
            p_sub = p.loc[p.scenario==scenarios[i]]
            pos.plot(p_sub['year'], p_sub['value','mean'], marker='o', label=scenario_names[i])
            pos.fill_between(p_sub['year'], p_sub['value','2.5%'], p_sub['value','97.5%'], alpha=0.2)
        pos.grid()
        pos.set_xticks(p_sub.year.unique())
        pos.set_xticklabels(p_sub.year.unique(),fontsize=15)
        pos.tick_params(axis='y', labelsize=15)
        if metrics.index(metric)==0:
            pos.set_ylim(0,p.loc[p.scenario=='baseline']['value','97.5%'].max()*1.1)
            pos.set_title(cause_titles[causes.index(cause)],size=20)
            if causes.index(cause)==0:
                pos.set_ylabel('Percent exposed\n',size=20)
                pos.legend(#['Baseline','+Wasting Treatment','+SQ-LNS','+Zinc supplementation'], 
                       fontsize=15)
        if metric=='percent_reduction':
            pos.set_ylim(-2,50)
            if causes.index(cause)==0:
                pos.set_ylabel('Percent reduction\nin exposure\nrelative to baseline\n',size=20)

plt.subplots_adjust(right=len(causes), top=len(metrics))
#plt.legend(['Baseline','Alternative scenario 1','Alternative scenario 2'], bbox_to_anchor=[0.25,4.15], fontsize=20)
plt.suptitle('Simulated intervention impacts on acute malnutrition\ncross-sectional exposure among children six to 11 months\n(with 95% uncertainty intervals)',
             y=2.7,x=1.5,size=25)

# Finding 2: incidence

In [ ]:
format_2026_table('incidence', 'value', 'baseline')

In [ ]:
format_2026_table('incidence', 'absolute_reduction', 'baseline')

In [ ]:
format_2026_table('incidence', 'percent_reduction', 'baseline')

In [ ]:
causes = ['mild_child_wasting_to_moderate_acute_malnutrition_event_count',
          'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count',
          #'total_mam',
          #'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count'
         ]

cause_titles = ['Moderate acute malnutrition\n',
                'Severe acute malnutrition\n',
                #'All transitions\ninto MAM\n',
                #'MAM cases from\nspontaneous\nSAM recovery'
               ]
metrics = ['value','absolute_reduction','percent_reduction']
metric_titles=['New cases per\n100,000 person-years\n',
              'Reduction in new cases\nper 100,000 person-years\nrelative to baseline\n',
              'Percent reduction\nin annual new cases\nrelative to baseline\n']

fig, axs = plt.subplots(len(metrics), len(causes),dpi=250)
for metric in metrics:
    data = calculate_wasting_values(output_dir, six_to_59_ages, ['year'], 
                                        'incidence', 
                                        metric, 
                                        'baseline')
    for cause in causes:
        p = data.loc[data.cause==cause]
        pos = axs[metrics.index(metric),causes.index(cause)]   
        for i in list(range(0,len(scenarios))):
            p_sub = p.loc[p.scenario==scenarios[i]]
            pos.plot(p_sub['year'], p_sub['value','mean'], marker='o', label=scenario_names[i])
            pos.fill_between(p_sub['year'], p_sub['value','2.5%'], p_sub['value','97.5%'], alpha=0.2)
        pos.grid()
        pos.set_xticks(p_sub.year.unique())
        pos.set_xticklabels(p_sub.year.unique(),fontsize=15)
        pos.tick_params(axis='y', labelsize=15)
        if causes.index(cause)==0:
            pos.set_ylabel(metric_titles[metrics.index(metric)],size=20)
            if metrics.index(metric)==0:
                pos.legend(#['Baseline','Alternative scenario 1','Alternative scenario 2'],
                      #bbox_to_anchor=[0.25,4.15], 
                       fontsize=15)
        if metrics.index(metric)==0:
            pos.set_title(cause_titles[causes.index(cause)],size=20)
            pos.set_ylim(0,p.loc[p.scenario=='baseline']['value','97.5%'].max()*1.1)
            
plt.subplots_adjust(right=len(causes), top=len(metrics))
#plt.legend(['Baseline','Alternative scenario 1','Alternative scenario 2'], bbox_to_anchor=[0.25,4.15], fontsize=20)
plt.suptitle('Simulated intervention impacts on acute malnutrition\nincidence rates among children six to 59 months\n(with 95% uncertainty intervals)',
             y=len(metrics)+.5,x=1,size=25)

# Finding 3: relative to alternative scenario 1

In [ ]:
relative_scenarios = ['baseline','sam_treatment','mam_treatment','sqlns','lbwsg_interventions']

causes = ['wasted','moderate_acute_malnutrition','severe_acute_malnutrition']
cause_titles = ['Acute malnutrition\n(MAM and SAM)\n',
               'Moderate acute\nmalnutrition\n',
               'Severe acute\nmalnutrition\n']

fig, axs = plt.subplots(len(relative_scenarios), len(causes),dpi=250)
for relative_scenario in relative_scenarios:
    data = calculate_wasting_values(output_dir, six_to_59_ages, ['year'], 
                                        'prevalence', 
                                        'percent_reduction', 
                                        relative_scenario)
    for cause in causes:
        p = data.loc[data.cause==cause]
        pos = axs[relative_scenarios.index(relative_scenario),causes.index(cause)]  
        scens = scenarios[scenarios.index(relative_scenario):len(scenarios)]
        
        for scenario in scens:
            p_sub = p.loc[p.scenario==scenario]
            pos.plot(p_sub['year'], p_sub['value','mean'], marker='o', 
                     label=scenario_names[scenarios.index(scenario)],
                    color=scenario_colors[scenarios.index(scenario)])
            pos.fill_between(p_sub['year'], p_sub['value','2.5%'], p_sub['value','97.5%'], 
                             alpha=0.2, color=scenario_colors[scenarios.index(scenario)])
        pos.grid()
        pos.set_xticks(p_sub.year.unique())
        pos.set_xticklabels(p_sub.year.unique(),fontsize=15)
        pos.tick_params(axis='y', labelsize=15)
        pos.set_ylim(-2,43)
        if relative_scenarios.index(relative_scenario)==0:
            pos.set_title(cause_titles[causes.index(cause)],size=20)
        if causes.index(cause)==0:
            pos.set_ylabel(f'Percent reduction in\nexposure relative to\n{scenario_names[relative_scenarios.index(relative_scenario)]}'
                               ,size=20)
            pos.legend(fontsize=15, loc='upper left')

plt.subplots_adjust(right=len(causes), top=len(relative_scenarios))
plt.suptitle('Percent reduction in simulated acute malnutrition\ncross sectional exposure among children six to 59 months\n(with 95% uncertainty intervals)',
             y=len(relative_scenarios) + 0.7,x=1.5,size=25)

In [ ]:
relative_scenarios = ['baseline','sam_treatment','mam_treatment','sqlns','lbwsg_interventions']
causes = ['mild_child_wasting_to_moderate_acute_malnutrition_event_count',
          'moderate_acute_malnutrition_to_severe_acute_malnutrition_event_count',
         ]

cause_titles = ['Moderate acute malnutrition\n',
                'Severe acute malnutrition\n',
               ]

fig, axs = plt.subplots(len(relative_scenarios), len(causes),dpi=250)
for relative_scenario in relative_scenarios:
    data = calculate_wasting_values(output_dir, six_to_59_ages, ['year'], 
                                        'incidence', 
                                        'percent_reduction', 
                                        relative_scenario)
    for cause in causes:
        p = data.loc[data.cause==cause]
        pos = axs[relative_scenarios.index(relative_scenario),causes.index(cause)]  
        scens = scenarios[scenarios.index(relative_scenario):len(scenarios)]
        
        for scenario in scens:
            p_sub = p.loc[p.scenario==scenario]
            pos.plot(p_sub['year'], p_sub['value','mean'], marker='o', 
                     label=scenario_names[scenarios.index(scenario)],
                    color=scenario_colors[scenarios.index(scenario)])
            pos.fill_between(p_sub['year'], p_sub['value','2.5%'], p_sub['value','97.5%'], 
                             alpha=0.2, color=scenario_colors[scenarios.index(scenario)])
        pos.grid()
        pos.set_xticks(p_sub.year.unique())
        pos.set_xticklabels(p_sub.year.unique(),fontsize=15)
        pos.tick_params(axis='y', labelsize=15)
        pos.set_ylim(-5,38)
        if relative_scenarios.index(relative_scenario)==0:
            pos.set_title(cause_titles[causes.index(cause)],size=20)
        if causes.index(cause)==0:
            pos.set_ylabel(f'Percent reduction in\nnew cases relative to\n{scenario_names[relative_scenarios.index(relative_scenario)]}'
                               ,size=20)
            pos.legend(fontsize=15, loc='upper left')

plt.subplots_adjust(right=len(causes), top=len(relative_scenarios))
plt.suptitle('Percent reduction in simulated acute malnutrition\nincidence rates among children six to 59 months\n(with 95% uncertainty intervals)',
             y=len(relative_scenarios) + 0.5,x=1,size=25)

# Finding 4: stunting exposures

In [ ]:
# stunting reduction
def calculate_stunting_values(output_dir, ages, strata_cols, metric, 
                             relative_scenario):
    data = pd.read_csv(output_dir + 'stunting_state_person_time.csv').drop(columns=['Unnamed: 0'])
    stunted = data.loc[data.cause.isin(['cat1','cat2'])]
    groupby_cols = ['input_draw','scenario','cause']
    stunted = (stunted.groupby([c for c in stunted.columns if c != 'cause' and c != 'value']).sum())
    stunted['cause'] = 'stunted'
    data = pd.concat([data,stunted.reset_index()],ignore_index=True)
    data = data.loc[data.age.isin(ages)]
    data = data.groupby(['input_draw','scenario'] + strata_cols + ['cause']).sum()
    data = data / data.groupby(['input_draw','scenario'] + strata_cols).sum() * 100
    if metric=='value':
        data = (data.groupby(['scenario','cause'] + strata_cols)
                    .describe(percentiles=[0.025,0.975]))
    else:
        data_baseline = (data.reset_index().loc[data.reset_index().scenario==relative_scenario]
                   .set_index(['input_draw','cause'] + strata_cols)
                   .drop(columns='scenario'))
        if metric=='percent_reduction':
            data = ((data_baseline - data.reset_index().set_index(['input_draw','cause'] + strata_cols + ['scenario']))
                    / data_baseline) * 100
        elif metric=='absolute_reduction':
            data = data_baseline - data.reset_index().set_index(['input_draw','cause'] + strata_cols + ['scenario'])
            
        data = (data.reset_index().groupby(['scenario','cause'] + strata_cols)
                .describe(percentiles=[0.025,0.975])).drop(columns='input_draw')
        
    return data[['value']].reset_index()

In [ ]:
def plot_values_by_year(data,
                          title, 
                          ylabel,
                          legend=True):
    plt.figure(dpi=120)
    p = data.copy()
    for i in ['baseline','sam_treatment','mam_treatment','sqlns','lbwsg_interventions','zinc_supplementation']:
        p_sub = p.loc[p.scenario==i]
        plt.plot(p_sub['year'], p_sub['value','mean'], marker='o', label=i)
        plt.fill_between(p_sub['year'], p_sub['value','2.5%'], p_sub['value','97.5%'], alpha=0.2)
    if legend==True:
        plt.legend(#['Baseline',
                #'Wasting treatment scale-up',
                #'SQ-LNS and wasting treatment scale-up'],
              fontsize=13)
    plt.xticks(p.year.unique(),size=14)
    plt.yticks(size=14)
    plt.ylabel(ylabel,size=14)
    plt.ylim(0,p['value','97.5%'].max()*1.12)
    plt.grid()
    plt.title(title, size=18)

In [ ]:
b = calculate_stunting_values(output_dir, six_to_59_ages, ['year'], 'value', 
                             'baseline')
b = b.loc[b.year==2026]
b['result'] = (b['value','mean'].round(1).astype(str)
              + ' ('
              + b['value','2.5%'].round(1).astype(str)
              + ', '
              + b['value','97.5%'].round(1).astype(str)
              + ')')
b.loc[b.cause=='cat1'].drop(columns='value')#.pivot_table(index='cause',columns='scenario',values='result')

In [ ]:
b = calculate_stunting_values(output_dir, six_to_59_ages, ['year'], 'percent_reduction', 
                             'baseline')
b = b.loc[b.year==2026]
b['result'] = (b['value','mean'].round(1).astype(str)
              + ' ('
              + b['value','2.5%'].round(1).astype(str)
              + ', '
              + b['value','97.5%'].round(1).astype(str)
              + ')')
b.loc[b.cause=='stunted'].drop(columns='value')#.pivot_table(index='cause',columns='scenario',values='result')

In [ ]:
causes = ['stunted',
          'cat2',
          'cat1',]

cause_titles = ['Stunted (moderate\nand severe)\n',
                'Moderate stunting\n',
                'Severe stunting\n'
               ]

metrics = ['value','percent_reduction']

scens = ['baseline','sqlns','lbwsg_interventions']

fig, axs = plt.subplots(len(metrics), len(causes),dpi=250)
for metric in metrics:
    data = calculate_stunting_values(output_dir, six_to_59_ages, ['year'], 
                                        metric, 
                                        'baseline')
    for cause in causes:
        p = data.loc[data.cause==cause]
        pos = axs[metrics.index(metric),causes.index(cause)]   
        for i in list(range(0,len(scens))):
            p_sub = p.loc[p.scenario==scens[i]]
            pos.plot(p_sub['year'], p_sub['value','mean'], marker='o', label=scenario_names[scenarios.index(scens[i])],
                    c = scenario_colors[scenarios.index(scens[i])])
            pos.fill_between(p_sub['year'], p_sub['value','2.5%'], p_sub['value','97.5%'], alpha=0.2,
                            color = scenario_colors[scenarios.index(scens[i])])
        pos.grid()
        pos.set_xticks(p_sub.year.unique())
        pos.set_xticklabels(p_sub.year.unique(),fontsize=15)
        pos.tick_params(axis='y', labelsize=15)
        if metrics.index(metric)==0:
            pos.set_ylim(0,p.loc[p.scenario=='baseline']['value','97.5%'].max()*1.1)
            pos.set_title(cause_titles[causes.index(cause)],size=20)
            if causes.index(cause)==0:
                pos.set_ylabel('Percent exposed\n',size=20)
                pos.legend(#['Baseline','Alternative scenario 1','Alternative scenario 2'],
                       fontsize=15)
        else:
            pos.set_ylim(-2,23)
            if causes.index(cause)==0:
                pos.set_ylabel('Percent reduction in exposure\nrelative to baseline\n',size=20)


plt.subplots_adjust(right=len(causes), top=len(metrics))
plt.suptitle('Simulated intervention impacts on child stunting\nexposure among children six to 59 months\n(with 95% uncertainty intervals)',
             y=2.6,x=1.5,size=25)

# Finding 5: mortality and morbidity outcomes

In [ ]:
def calculate_mortality_values(output_dir, ages, strata_cols, metric, 
                             relative_scenario, measure):
    if measure == 'dalys':
        ylls = pd.read_csv(output_dir + 'ylls.csv').drop(columns=['Unnamed: 0','measure'])
        ylds = pd.read_csv(output_dir + 'ylds.csv').drop(columns=['Unnamed: 0','measure'])
        data = pd.concat([ylls,ylds]).groupby([c for c in ylls.columns if c != 'value' and c != 'cause']).sum().reset_index()
    else:
        data = pd.read_csv(output_dir + f'{measure}.csv').drop(columns=['Unnamed: 0'])
    pt = pd.read_csv(output_dir + 'wasting_state_person_time.csv').drop(columns='Unnamed: 0')
    groupby_cols = ['input_draw','scenario']
    data = data.loc[data.age.isin(ages)]
    pt = pt.loc[pt.age.isin(ages)]
    data = (data.groupby(['input_draw','scenario'] + strata_cols).sum()
            / pt.groupby(['input_draw','scenario'] + strata_cols).sum()
            * 100_000)
    if metric=='val':
        data = (data.groupby(['scenario'] + strata_cols)
                    .describe(percentiles=[0.025,0.975]))
    else:
        data_baseline = (data.reset_index().loc[data.reset_index().scenario==relative_scenario]
                   .set_index(['input_draw'] + strata_cols)
                   .drop(columns='scenario'))
        if metric=='percent_reduction':
            data = ((data_baseline - data.reset_index().set_index(['input_draw'] + strata_cols + ['scenario']))
                    / data_baseline) * 100
        elif metric=='absolute_reduction':
            data = data_baseline - data.reset_index().set_index(['input_draw'] + strata_cols + ['scenario'])
            
        data = (data.reset_index().groupby(['scenario'] + strata_cols)
                .describe(percentiles=[0.025,0.975])).drop(columns='input_draw')
        
    return data[['value']].reset_index()

In [ ]:
b = calculate_mortality_values(output_dir, ages, ['year'], 'absolute_reduction', 
                             'baseline', 'dalys')
b = b.loc[b.year==2026]
b['result'] = (b['value','mean'].round(0).astype(int).astype(str)
              + ' ('
              + b['value','2.5%'].round(0).astype(int).astype(str)
              + ', '
              + b['value','97.5%'].round(0).astype(int).astype(str)
              + ')')
b['scenario_order'] = np.where(b.scenario=='baseline',0,
                                  np.where(b.scenario=='sam_treatment',1,
                                          np.where(b.scenario=='mam_treatment',2,
                                                  np.where(b.scenario=='sqlns',3,
                                                          np.where(b.scenario=='lbwsg_interventions',4,5)))))
b=b.sort_values(by='scenario_order')
b.drop(columns='value')

In [ ]:
b = calculate_mortality_values(output_dir, ages, ['year'], 'val', 
                             'baseline', 'ylls')
b = b.loc[b.year==2026]
b['result'] = (b['value','mean'].round(0).astype(int).astype(str)
              + ' ('
              + b['value','2.5%'].round(0).astype(int).astype(str)
              + ', '
              + b['value','97.5%'].round(0).astype(int).astype(str)
              + ')')
b['scenario_order'] = np.where(b.scenario=='baseline',0,
                                  np.where(b.scenario=='sam_treatment',1,
                                          np.where(b.scenario=='mam_treatment',2,
                                                  np.where(b.scenario=='sqlns',3,
                                                          np.where(b.scenario=='lbwsg_interventions',4,5)))))
b=b.sort_values(by='scenario_order')
b.drop(columns='value')

In [ ]:
measures = ['deaths',
          'dalys',
          'ylls',
         'ylds']
measure_titles = ['Deaths\n',
                 'Disability adjusted\nlife years (DALYs)\n',
                 'Years of life\nlost (YLLs)\n',
                 'Years lived with\ndisability (YLDs)\n']

metrics = ['val','absolute_reduction','percent_reduction']
metric_titles = ['Rate per 100,000\nperson-years\n',
                 'Rate reduction per\n100,000 person-years\nrelative to baseline\n',
                 'Percent reduction\nrelative to baseline\n'
                ]

fig, axs = plt.subplots(len(metrics), 
                        len(measures),
                        dpi=250)
for metric in metrics:
    for measure in measures:
        data = calculate_mortality_values(output_dir, ages, ['year'], metric, 
                             'baseline', measure)
        pos = axs[metrics.index(metric),measures.index(measure)]   
        for i in list(range(0,len(scenarios))):
            p_sub = data.loc[data.scenario==scenarios[i]]
            pos.plot(p_sub['year'], p_sub['value','mean'], marker='o', label=scenario_names[i])
            pos.fill_between(p_sub['year'], p_sub['value','2.5%'], p_sub['value','97.5%'], alpha=0.2)
        pos.grid()
        pos.set_xticks(p_sub.year.unique())
        pos.set_xticklabels(p_sub.year.unique(),fontsize=15)
        pos.tick_params(axis='y', labelsize=15)
        if metrics.index(metric)==0:
            pos.set_title(measure_titles[measures.index(measure)],size=20)
            pos.set_ylim(0,data.loc[data.scenario=='baseline']['value','97.5%'].max()*1.1)
            if measures.index(measure)==0:
                pos.legend(fontsize=15)
        if measures.index(measure)==0:
            pos.set_ylabel(metric_titles[metrics.index(metric)],size=20)
        if metric=='percent_reduction':
            pos.set_ylim(-2,32)
       

plt.subplots_adjust(right=len(measures), top=len(metrics))
plt.suptitle('Simulated morbidity and mortality outcomes\namong children aged six to 59 months\n(with 95% uncertainty intervals)',
             y=3.6,x=2,size=25)

In [ ]:
baseline_vals = [15, 48.8, 73.1, 70]
baseline_lcls = [10, 37.4, 58.5, 64]
baseline_ucls = [20, 60.4, 87.7, 76]
targets = [70, 70, 75, 75]
titles = ['MAM coverage',
         'SAM coverage',
         'MAM efficacy',
         'SAM efficacy']
num1s = [0,0,1,1]
num2s = [0,1,0,1]

years = list(range(2022,2028))
fig, axs = plt.subplots(2, 
                        2,
                        dpi=250)

for i in list(range(0,len(baseline_vals))):

    axs[num1s[i],num2s[i]].plot(years,[baseline_vals[i]]*len(years))
    axs[num1s[i],num2s[i]].fill_between(years, [baseline_lcls[i]]*len(years), [baseline_ucls[i]]*len(years),alpha=0.2)
    axs[num1s[i],num2s[i]].plot([2022,2023,2026,2027],[baseline_vals[i],
                                                       baseline_vals[i],
                                                       targets[i],
                                                       targets[i]], linestyle='dashed')
    axs[num1s[i],num2s[i]].fill_between([2022,2023,2026,2027],[baseline_lcls[i],
                                                       baseline_lcls[i],
                                                       targets[i],
                                                       targets[i]],
                                        [baseline_ucls[i],
                                                       baseline_ucls[i],
                                                       targets[i],
                                                       targets[i]],alpha=0.2)
    axs[num1s[i],num2s[i]].grid()
    axs[num1s[i],num2s[i]].set_ylim(0,100)
    axs[num1s[i],num2s[i]].set_title(titles[i],size=15)

plt.legend(['Baseline','Alternative'],bbox_to_anchor=[1.5,2.2],fontsize=15)
plt.subplots_adjust(right=2, top=2)

In [ ]:
baseline_vals=[0]
baseline_ucls=[0]
baseline_lcls=[0]
targets=[90]
#plt.figure(dpi=250)
for i in [0]:
    plt.plot(years,[baseline_vals[i]]*len(years))
    plt.fill_between(years, [baseline_lcls[i]]*len(years), [baseline_ucls[i]]*len(years),alpha=0.2)
    plt.plot([2022,2023,2026,2027],[baseline_vals[i],
                                                       baseline_vals[i],
                                                       targets[i],
                                                       targets[i]], linestyle='dashed')
    plt.fill_between([2022,2023,2026,2027],[baseline_lcls[i],
                                                       baseline_lcls[i],
                                                       targets[i],
                                                       targets[i]],
                                        [baseline_ucls[i],
                                                       baseline_ucls[i],
                                                       targets[i],
                                                       targets[i]],alpha=0.2)
plt.grid()
plt.ylim(-5,105)
plt.title('SQ-LNS coverage')
plt.legend(['Baseline','Alternative'])

In [ ]:
data = calculate_wasting_values(output_dir, six_to_59_ages, ['year'], 
                                    'prevalence', 
                                    metric, 
                                    'baseline')
cause=='severe_acute_malnutrition'
#for cause in causes:
p = data.loc[data.cause==cause]
pos = axs[0,0]   
for i in list(range(0,len(scenarios))):
    p_sub = p.loc[p.scenario==scenarios[i]]
    pos.plot(p_sub['year'], p_sub['value','mean'], marker='o', label=scenario_names[i])
    pos.fill_between(p_sub['year'], p_sub['value','2.5%'], p_sub['value','97.5%'], alpha=0.2)
pos.grid()

In [ ]:
data = calculate_wasting_values(output_dir, six_to_59_ages, ['year'], 
                                    'prevalence', 
                                     metric, 
                                    'baseline')
p = data.loc[data.cause=='wasted']
for scenario in scenarios:
    p_sub = p.loc[p.scenario==scenario]
    #plt.scatter(p_sub.year, p_sub['value','mean'],marker='o')
    p_sub_2022 = p_sub.loc[p_sub.year==2022]
    p_sub_2022['year'] = 2022.4
    p_sub_2025 = p_sub.loc[p_sub.year==2026]
    p_sub_2025['year'] = 2025.6
    p_sub = pd.concat([p_sub,p_sub_2022,p_sub_2025]).sort_values(by='year')
    p_sub_2022 = p_sub.loc[p_sub.year==2022]
    p_sub_2022['year'] = 2021.5
    p_sub_2025 = p_sub.loc[p_sub.year==2026]
    p_sub_2025['year'] = 2026.5
    p_sub = pd.concat([p_sub,p_sub_2022,p_sub_2025]).sort_values(by='year')
    plt.fill_between(p_sub.year,p_sub['value','2.5%'],p_sub['value','97.5%'],alpha=0.2, 
                     color=scenario_colors[scenarios.index(scenario)])
    plt.plot(p_sub.year,p_sub['value','mean'])
plt.xticks(data.year.unique())
plt.grid()

In [ ]:
p_sub

In [ ]:
data.cause.unique()

In [ ]:
data